In [1]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
from os import listdir

#for a particle on-shell:
# E = mT cosh y 
# pz = mT sinh y
#mT2 = m2 + pt2
# rapidity in the ion-at-rest frame, with the photon direction is 
# y = 0.5 * ln( (E+pz)/(E-pz))
#rapidity of the target in the lab frame = 0 (at rest)
# y = y* + ycm
#bcm = az + bz/ a0 + b0 
#bcm = photon_z / Nu + m_{p}
#ycm = 0.5 * ln( 1+bcm/1-bcm)
#energy of the photon = Nu
#momentum of the photon (magnitude) = sqrt(Nu2 + Q2) 
#q2 = -Q2. (four-momentum transfer)

Welcome to JupyROOT 6.15/01


In [2]:
class particle:
    def __init__(self, pid, fourvector, virtual_photon,PhiPQ):
        #work in frame that is aligned with photon
        #the photon is aligned with z-direction and has 4-momentum q= (0,0,sqrt(Nu2+Q2),Nu) (note q2 = -Q2)        
        Nu = virtual_photon.E()
        Q2 = -virtual_photon.M2()
        proton = ROOT.TLorentzVector()
        proton.SetPxPyPzE(0,0,0, 0.938)
        W = (virtual_photon + proton).M()
        
        photon_pz = np.sqrt(Nu*Nu+Q2) #direction is positive by definition
        self.bcm = photon_pz/(Nu + 0.938)#photon-nucleon center-of-mass velocity 
        self.ycm = 0.5*np.log(( 1+self.bcm)/(1-self.bcm)) #photon-nucleon center-of-mass rapidity
        
        self.LorentzVector = fourvector
        self.E = fourvector.E() #energy in lab frame
        self.vector = fourvector.Vect()
        self.Pt = self.vector.Perp(virtual_photon.Vect().Unit()) #pT with respect to photon direction
        self.Pl  = self.vector.Dot(virtual_photon.Vect().Unit()) #pL with respect to photon direction (in lab frame)
        self.y =  0.5*np.log( (self.E+self.Pl)/(self.E-self.Pl)) #rapidity in lab frame
        self.mT = np.sqrt(fourvector.M2() + self.Pt*self.Pt)
        self.y_star = self.y - self.ycm
        self.Pl_star = self.mT*np.sinh(self.y_star)
        self.Xf = 2.0*self.Pl_star/W 
        self.pid = pid
        self.Zh = self.E/Nu
        self.PhiPQ = PhiPQ*np.pi/180.0
        
        
    def print_properties(self):
        print 'Hello, let me introduce myself, i am particle pid = ' , self.pid, ' with index ', self.index, ', from event  #', self.ievt, ' Nu and W', self.Nu, ' ' , self.W
        print 'zh = ', self.Zh, ' phi_pq= ', self.PhiPQ, ' theta_pq=' , self.ThetaPQ, 'E = ', self.E, ' xf', self.Xf,'Pt ', self.Pt, ' Pl= ', self.Pl, ' rapidity=' ,  self.y
        print 'pid = ' , self.pid

In [6]:

def getDataframes(filename, Target=1,maxevents=1e9):
    dphi = np.array([])
    ParticlesFromPrevious = []
    try:
        myfile = TFile.Open('%s'%filename,'READ')
    except:
        print("could not open file")

        
    myfile.Print()
    mytree = myfile.Get('tree_data')
    e_tree = myfile.Get('e_rec')
    
    
    print filename, ' has ', mytree.GetEntries(), ' entries'
    tupla = {}
    tupla['dphi'] = []
    tupla['dy'] = []
    tupla['z'] = [] 
    tupla['z_asso'] = []
    tupla['xf'] = []
    tupla['xf_asso'] = []
    tupla['y'] = []
    tupla['y_star'] = []

    tupla['y_asso'] = []
    tupla['pid'] = []
    tupla['pid_asso'] = []
    tupla['phi_pq'] = []
    tupla['phi_pq_asso'] = []
    tupla['mass'] = []
    tupla['dipion_pt'] = []
    tupla['missing_mass'] = []
    tupla['t']  = []
    tupla['Q2'] = [] 
    tupla['Nu'] = []
    tupla['Xb'] = []
    tupla['u']  = []

    tupla_mix = {}
    tupla_mix['dphi'] = []
    tupla_mix['dy'] = []
    tupla_mix['z'] = []
    tupla_mix['z_asso'] = []
    tupla_mix['xf'] = []
    tupla_mix['xf_asso'] = []
    tupla_mix['y'] = []
    tupla_mix['y_star'] = []

    tupla_mix['y_asso'] = []
    tupla_mix['pid'] = []
    tupla_mix['pid_asso'] = []
    tupla_mix['phi_pq']   = []
    tupla_mix['phi_pq_asso'] = []
    tupla_mix['mass'] = []
    tupla_mix['dipion_pt'] = []
    tupla_mix['missing_mass'] = []
    tupla_mix['t']  = []
    tupla_mix['Q2'] = []
    tupla_mix['Nu'] = []
    tupla_mix['Xb'] = []
    tupla_mix['u']  = []
    
    tupla_trigger = {}
    tupla_trigger['xf'] = []
    tupla_trigger['xf_default'] = []
    tupla_trigger['z']  = []
    tupla_trigger['y']  = []
    tupla_trigger['y_star']  = []
    tupla_trigger['bcm'] = []
    tupla_trigger['Q2'] = []
    tupla_trigger['Xb'] = []
    tupla_trigger['Nu'] = []
    tupla_trigger['phi_pq'] = []
    tupla_trigger['TargType'] = []
    tupla_trigger['mass'] = []

    start = time.time()

    for ievt  in range(mytree.GetEntries()):
        mytree.GetEntry(ievt)   
        e_tree.GetEntry(ievt)
        if mytree.W<2.05 or mytree.Q2<1.0: continue
        if ievt>maxevents: break
        if mytree.TargType!=Target: continue

        #get electron momentum:
        Pe = np.sqrt(e_tree.Pex*e_tree.Pex + e_tree.Pey*e_tree.Pey+ e_tree.Pez*e_tree.Pez)
        scattered_e = ROOT.TLorentzVector()
        scattered_e.SetPxPyPzE(e_tree.Pex, e_tree.Pey, e_tree.Pez, Pe)
        incoming_e = ROOT.TLorentzVector()
        incoming_e.SetPxPyPzE(0,0,5.014,5.014)
        virtual_photon  = incoming_e - scattered_e 
        virtual_photon_unitvector = virtual_photon.Vect().Unit()
        proton = ROOT.TLorentzVector()
        proton.SetPxPyPzE(0,0,0, 0.938)
        
        #photon-nucleon center-of-mass energy and rapidity
        #the photon is aligned with z-direction and has 4-momentum q= (0,0,sqrt(Nu2+Q2),Nu) (note q2 = -Q2)
        
        W = mytree.W
        Nu = mytree.Nu
        #print 'W ',  W , 'electron W ', e_tree.W
        #print (virtual_photon + proton).M()
        particles = []
        #print ' Entering main loop over particles'
        for i in range(len(mytree.pid)):
            if abs(mytree.pid[i]) !=211: continue
            i_lv = ROOT.TLorentzVector()    
            i_lv.SetPxPyPzE(mytree.Px[i],mytree.Py[i],mytree.Pz[i],mytree.Zh[i]*Nu)
            i_part = particle(mytree.pid[i], i_lv, virtual_photon, mytree.PhiPQ[i] )
            particles.append(i_part)                

            
            tupla_trigger['TargType'].append(mytree.TargType)
            tupla_trigger['xf'].append(i_part.Xf)
            tupla_trigger['xf_default'].append(mytree.Xf[i])
            tupla_trigger['z'].append(i_part.Zh)
            tupla_trigger['y'].append(i_part.y)
            tupla_trigger['y_star'].append(i_part.y_star)
            tupla_trigger['bcm'].append(i_part.bcm)
            tupla_trigger['phi_pq'].append(i_part.PhiPQ)
            tupla_trigger['mass'].append(i_part.LorentzVector.M())
            tupla_trigger['Q2'].append(mytree.Q2)
            tupla_trigger['Xb'].append(mytree.Xb)
            tupla_trigger['Nu'].append(mytree.Nu)

            #print 'mytree.Pt[i] ' , mytree.Pt[i], ' check: ' ,i_part.Vector.Perp(virtual_photon_unitvector)
            for j in range(len(mytree.pid)): 
                if i==j: continue
                if abs(mytree.pid[j]) !=211: continue
                j_lv = ROOT.TLorentzVector()    
                j_lv.SetPxPyPzE(mytree.Px[j],mytree.Py[j],mytree.Pz[j],mytree.Zh[j]*Nu)
                j_part = particle(mytree.pid[j], j_lv, virtual_photon, mytree.PhiPQ[j] )
                    
                dphi = abs(ROOT.TVector2.Phi_mpi_pi(i_part.PhiPQ-j_part.PhiPQ))
                dy = i_part.y-j_part.y
                deta = 0
                ### FILLING CORRELATIONS

                dipion = i_part.LorentzVector+j_part.LorentzVector
                X = (virtual_photon + proton - dipion) #unobserved hadronic system
                
                tupla['dphi'].append(dphi)
                tupla['dy'].append(dy)
                tupla['z'].append(i_part.Zh)
                tupla['z_asso'].append(j_part.Zh)
                tupla['xf'].append(i_part.Xf)
                tupla['xf_asso'].append(j_part.Xf)
                tupla['y'].append(i_part.y)
                tupla['y_star'].append(i_part.y_star)
                tupla['y_asso'].append(j_part.y)
                tupla['pid'].append(i_part.pid)
                tupla['pid_asso'].append(j_part.pid)
                tupla['phi_pq'].append(i_part.PhiPQ)
                tupla['phi_pq_asso'].append(j_part.PhiPQ)
                tupla['mass'].append(dipion.M())
                tupla['dipion_pt'].append( dipion.Vect().Perp(virtual_photon_unitvector))
                tupla['missing_mass'].append(X.M())
                tupla['t'].append( -(virtual_photon- dipion).M2())
                tupla['Q2'].append(mytree.Q2)
                tupla['Xb'].append(mytree.Xb)
                tupla['Nu'].append(mytree.Nu)
                tupla['u'].append(-(scattered_e-proton).M2())
                
                #end loop over secondary loop    
            #print '//////// Entering mixed event correlations with # ', len(ParticlesFromPrevious) , ' paticles in previous event'
            if i_part.Zh > 0.4: #do mixing
                for mixparticle in ParticlesFromPrevious:
                    dipion = i_part.LorentzVector+mixparticle.LorentzVector

                    X = (virtual_photon + proton - dipion)

                    dphi = abs(ROOT.TVector2.Phi_mpi_pi(i_part.PhiPQ-mixparticle.PhiPQ))
                    dy = i_part.y-mixparticle.y
                    deta = 0#i_part.ThetaPQ-mixparticle.ThetaPQ
                    tupla_mix['dphi'].append(dphi)
                    tupla_mix['dy'].append(dy)
                    tupla_mix['z'].append(i_part.Zh)
                    tupla_mix['z_asso'].append(mixparticle.Zh)
                    tupla_mix['xf'].append(i_part.Xf)
                    tupla_mix['xf_asso'].append(mixparticle.Xf)
                    tupla_mix['y'].append(i_part.y)
                    tupla_mix['y_star'].append(i_part.y_star)
                    tupla_mix['y_asso'].append(mixparticle.y)
                    tupla_mix['pid'].append(i_part.pid)
                    tupla_mix['pid_asso'].append(mixparticle.pid)
                    tupla_mix['phi_pq'].append(i_part.PhiPQ)
                    tupla_mix['phi_pq_asso'].append(mixparticle.PhiPQ)
                    tupla_mix['mass'].append(dipion.M())    
                    tupla_mix['dipion_pt'].append( dipion.Vect().Perp(virtual_photon_unitvector))
                    tupla_mix['missing_mass'].append(X.M())
                    tupla_mix['t'].append(-(virtual_photon- dipion).M2())
                    tupla_mix['Q2'].append(mytree.Q2)
                    tupla_mix['Xb'].append(mytree.Xb)
                    tupla_mix['Nu'].append(mytree.Nu)

                    tupla_mix['u'].append( -(scattered_e-proton).M2())
                    
                    
        ParticlesFromPrevious = particles
        #print ' going for next event'    
        #print ' particles in event', len(particles
        ##end loop over events correlations    
    end = time.time()
    print 'Processed in',  end-start, 'seconds'
    
    df = pd.DataFrame(tupla)
    df_mix= pd.DataFrame(tupla_mix)
    df_trigger = pd.DataFrame(tupla_trigger)
    print 'Number of triggers with z>0.4,  ', df.query('z>0.4').shape[0]
    print 'Number of pairs with z>0.4, ' ,df_trigger.query('z>0.4').shape[0] 
    myfile.Close()
    return df, df_mix, df_trigger

In [ ]:
#for a particle on-shell:
# E = mT cosh y 
# pz = mT sinh y
#mT2 = m2 + pt2
# rapidity in the ion-at-rest frame, with the photon direction is 
# y = 0.5 * ln( (E+pz)/(E-pz))
#rapidity of the target in the lab frame = 0 (at rest)
# y = y* + ycm
#bcm = az + bz/ a0 + b0 
#bcm = photon_z / Nu + m_{p}
#ycm = 0.5 * ln( 1+bcm/1-bcm)
#energy of the photon = Nu
#momentum of the photon (magnitude) = sqrt(Nu2 + Q2) 
#q2 = -Q2. (four-momentum transfer)

In [3]:
def transformToPhotonFrame(fourvector,virtual_photon):
    bcm = sqrt(virtual_photon.E()*virtual_photon.E() - virtual_photon.M())
    ycm = 0.5*np.ln(( 1+bcm)/(1-bcm)) 
    virtual_photon_unitvector = virtual_photon.Vect().Unit()
    vector = fourvector.Vect()
    pT = vector.Perp(virtual_photon_unitvector)
    mT = np.sqrt(fourvector.M2() + pT*pT)
    pL = vector.Dot(virtual_photon_unitvector)
    p = np.sqrt(pT*pT+pL*pL)
    y = 0.5*np.ln((E+pL)/(E-pL))
    y_star = y - ycm
    pz_star = mT*np.sinh(y_star) 
    xf = 2.0*pz_star/W
    
    return
    

In [4]:
df = {}

## Pb data

In [ ]:
path = '/mnt/c/Users/marratia/Linux/Pb_data/'

Files = listdir(path) 
df['Pb'],df['Pb_mix'], df['Pb_trigger'] = [None,None,None]
df['D_Pb'],df['D_Pb_mix'], df['D_Pb_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['Pb'] = pd.concat([ df['Pb'], pairs])
    df['Pb_mix'] = pd.concat([ df['Pb_mix'], pairs_mix])
    df['Pb_trigger'] = pd.concat([ df['Pb_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_Pb'] = pd.concat([ df['D_Pb'], pairs])
    df['D_Pb_mix'] = pd.concat([ df['D_Pb_mix'], pairs_mix])
    df['D_Pb_trigger'] = pd.concat([ df['D_Pb_trigger'], trigger])

## Fe data

In [7]:
path = '/mnt/c/Users/marratia/Linux/Fe_data/'
Files = listdir(path) 
df['Fe'],df['Fe_mix'], df['Fe_trigger'] = [None,None,None]
df['D_Fe'],df['D_Fe_mix'], df['D_Fe_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['Fe'] = pd.concat([ df['Fe'], pairs])
    df['Fe_mix'] = pd.concat([ df['Fe_mix'], pairs_mix])
    df['Fe_trigger'] = pd.concat([ df['Fe_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_Fe'] = pd.concat([ df['D_Fe'], pairs])
    df['D_Fe_mix'] = pd.concat([ df['D_Fe_mix'], pairs_mix])
    df['D_Fe_trigger'] = pd.concat([ df['D_Fe_trigger'], trigger])

/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41146.root  has  220529  entries


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]


Processed in 172.946122885 seconds
Number of triggers with z>0.4,   2148
Number of pairs with z>0.4,  9525
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41146.root  has  220529  entries
Processed in 144.504058123 seconds
Number of triggers with z>0.4,   3833
Number of pairs with z>0.4,  15127
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41147.root  has  208300  entries
Processed in 123.933987141 seconds
Number of triggers with z>0.4,   2122
Number of pairs with z>0.4,  9032
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41147.root  has  208300  entries
Processed in 125.307813883 seconds
Number of triggers with z>0.4,   3634
Number of pairs with z>0.4,  14653
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41148.root  has  182293  entries
Processed in 108.725160122 seconds
Number of triggers with z>0.4,   1905
Number of pairs with z>0.4,  7779
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41148.root  has  182293  entries
Processed in 121.444480181 seconds
Num

## C data

In [8]:
path = '/mnt/c/Users/marratia/Linux/C_data/'
Files = listdir(path) 
df['C'],df['C_mix'], df['C_trigger'] = [None,None,None]
df['D_C'],df['D_C_mix'], df['D_C_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['C'] = pd.concat([ df['C'], pairs])
    df['C_mix'] = pd.concat([ df['C_mix'], pairs_mix])
    df['C_trigger'] = pd.concat([ df['C_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_C'] = pd.concat([ df['D_C'], pairs])
    df['D_C_mix'] = pd.concat([ df['D_C_mix'], pairs_mix])
    df['D_C_trigger'] = pd.concat([ df['D_C_trigger'], trigger])

/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42011.root  has  195907  entries


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]


Processed in 118.819926023 seconds
Number of triggers with z>0.4,   2334
Number of pairs with z>0.4,  9942
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42011.root  has  195907  entries
Processed in 150.343938828 seconds
Number of triggers with z>0.4,   3106
Number of pairs with z>0.4,  12810
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42012.root  has  110697  entries
Processed in 66.3801980019 seconds
Number of triggers with z>0.4,   1320
Number of pairs with z>0.4,  5580
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42012.root  has  110697  entries
Processed in 101.802303076 seconds
Number of triggers with z>0.4,   1786
Number of pairs with z>0.4,  7297
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42013.root  has  183077  entries
Processed in 109.26796484 seconds
Number of triggers with z>0.4,   2290
Number of pairs with z>0.4,  9443
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42013.root  has  183077  entries
Processed in 118.452623129 seconds
Number of trigg

In [ ]:
df['Pb_trigger'].head()

In [ ]:
plt.hist2d(df['Pb_trigger'].query('z<0.3 and abs(xf)<1.0')['y_star'],df['Pb_trigger'].query('z<0.3 and abs(xf)<1.0')['xf'],bins=[100,100])

In [ ]:
plt.hist(df['Pb_trigger'].query('z<0.3')['y_star'],range=(-4,4),bins=100,alpha=0.4,normed=True)
plt.hist(df['Pb_trigger'].query('z>0.4')['y_star'],range=(-4,4),bins=100,alpha=0.4,normed=True)
plt.hist(df['Pb_trigger']['y_star'],range=(-4,4),bins=100,alpha=0.4,normed=True)

plt.show()

In [ ]:
plt.hist(df['Pb'].query('z>0.5')['dy'],range=(-4,4),bins=50,alpha=0.4,normed=True)
print df['Pb'].query('z>0.5').shape[0]

In [10]:
import root_pandas
from root_pandas import read_root
from root_pandas import to_root 

## Save dataframes to ROOT files

In [ ]:
to_root(df['Pb'],'Pairs_Pb.root', key='Pb')
to_root(df['Pb_mix'],'Pairs_Pb.root', key='Pb_mix',mode='a')
to_root(df['Pb_trigger'],'Pairs_Pb.root', key='Pb_trigger', mode='a')
to_root(df['D_Pb'],'Pairs_Pb.root', key='D_Pb',mode='a')
to_root(df['D_Pb_mix'],'Pairs_Pb.root', key='D_Pb_mix',mode='a')
to_root(df['D_Pb_trigger'],'Pairs_Pb.root', key='D_Pb_trigger', mode='a')

In [11]:
to_root(df['Fe'],'Pairs_Fe.root', key='Fe')
to_root(df['Fe_mix'],'Pairs_Fe.root', key='Fe_mix',mode='a')
to_root(df['Fe_trigger'],'Pairs_Fe.root', key='Fe_trigger', mode='a')
to_root(df['D_Fe'],'Pairs_Fe.root', key='D_Fe',mode='a')
to_root(df['D_Fe_mix'],'Pairs_Fe.root', key='D_Fe_mix',mode='a')
to_root(df['D_Fe_trigger'],'Pairs_Fe.root', key='D_Fe_trigger', mode='a')

In [12]:
to_root(df['C'],'Pairs_C.root', key='C')
to_root(df['C_mix'],'Pairs_C.root', key='C_mix',mode='a')
to_root(df['C_trigger'],'Pairs_C.root', key='C_trigger', mode='a')
to_root(df['D_C'],'Pairs_C.root', key='D_C',mode='a')
to_root(df['D_C_mix'],'Pairs_C.root', key='D_C_mix',mode='a')
to_root(df['D_C_trigger'],'Pairs_C.root', key='D_C_trigger', mode='a')